## Importing all the dependecies

In [ ]:
#!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text as text

## Load and read the data

In [ ]:
# Read the data
data = pd.read_csv('https://raw.githubusercontent.com/analyticsindiamagazine/MocksDatasets/main/NLP-movie-review.csv')

In [ ]:
# Top 5 rows
data.head()

,review,reaction
0,Service is friendly and inviting.,1
1,Awesome service and food.,1
2,Waitress was a little slow in service.,0
3,"Come hungry, leave happy and stuffed!",1
4,Horrible - don't waste your time and money.,0


In [ ]:
# data info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   review    1000 non-null   object
 1   reaction  1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [ ]:
# target values counts
data.reaction.value_counts()

1    500
0    500
Name: reaction, dtype: int64

## Splitting the data

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data.review.values, data.reaction.values, random_state = 42, test_size = .20)

In [ ]:
# train data shape
x_train.shape, y_train.shape

((800,), (800,))

In [ ]:
# test data shape
x_test.shape, y_test.shape

((200,), (200,))

## Loading, finetuning and training the BERT model

In [ ]:
# Load the BERT layers
bert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
# Neural network layers
#l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(outputs['pooled_output'])
# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
# model summury
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_12 (KerasLayer)    {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [ ]:
# compile and train the model
#from keras.optimizers.optimizer_v2.rmsprop
model.compile(loss= 'binary_crossentropy', optimizer = 'adam', metrics=['acc'])
model.fit(x_train, y_train, epochs=50, validation_split=0.10)

Epoch 1/50
23/23 [==============================] - 19s 392ms/step - loss: 0.6890 - acc: 0.5653 - val_loss: 0.6566 - val_acc: 0.6500
Epoch 2/50
23/23 [==============================] - 8s 370ms/step - loss: 0.6582 - acc: 0.6111 - val_loss: 0.6548 - val_acc: 0.6375
Epoch 3/50
23/23 [==============================] - 9s 375ms/step - loss: 0.6445 - acc: 0.6319 - val_loss: 0.6339 - val_acc: 0.6125
Epoch 4/50
23/23 [==============================] - 8s 369ms/step - loss: 0.6274 - acc: 0.6653 - val_loss: 0.6233 - val_acc: 0.6375
Epoch 5/50
23/23 [==============================] - 8s 363ms/step - loss: 0.6258 - acc: 0.6653 - val_loss: 0.6189 - val_acc: 0.6750
Epoch 6/50
23/23 [==============================] - 8s 357ms/step - loss: 0.6144 - acc: 0.6764 - val_loss: 0.6251 - val_acc: 0.6500
Epoch 7/50
23/23 [==============================] - 8s 356ms/step - loss: 0.6062 - acc: 0.6917 - val_loss: 0.5961 - val_acc: 0.7375
Epoch 8/50
23/23 [==============================] - 8s 354ms/step - loss: 0

## Prediction and evalution BERT model

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np

In [ ]:
pred = model.predict(x_test).flatten().round()
pred

7/7 [==============================] - 3s 316ms/step


array([1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1.,
       1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1.,
       1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1.,
       0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1.,
       0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0.,
       1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1.,
       1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1.], dtype=float32)

In [ ]:
# Confusion metrics
confusion_matrix(y_test, pred)

array([[85, 19],
       [16, 80]])

In [ ]:
# Classification report
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.84      0.82      0.83       104
           1       0.81      0.83      0.82        96

    accuracy                           0.82       200
   macro avg       0.82      0.83      0.82       200
weighted avg       0.83      0.82      0.83       200



In [ ]:
# accuracy score
accuracy_score(y_test, pred)

0.825